---
title: Pivot Tables
tags: [jupyter]
keywords: pandas
summary: "Using Pivot tables in pandas."
mlType: dataFrame
infoType: pandas
sidebar: pandas_sidebar
permalink: __AutoGenThis__
notebookfilename:  __AutoGenThis__
---

In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from pprint import pprint

# Padas Options

In [3]:
pd.set_option('max_rows', 8)

# I/O

In [4]:
titanic = sns.load_dataset('titanic')

In [5]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


# Long Hand

Let's say we want to find out the amount of survived by sex we can do something like this:

In [8]:
titanic.groupby('sex')[['survived']].mean()

,survived
sex,
female,0.742038
male,0.188908


If we want to go further with analysis we might want to:

- group by class and gender
- select survival
- apply a mean aggregate
- combine the resulting groups
- unstack the hierarchical index to reveal the hidden multidimensionality. 

In [10]:
titanic.groupby(['sex','class'])[['survived']].aggregate('mean').unstack()

survived                    
class      First    Second     Third
sex                                 
female  0.968085  0.921053  0.500000
male    0.368852  0.157407  0.135447

This is basically a **pivot table**

# Pandas Pivot Table

The full call signature of a pivot_table is:

```python
# call signature as of Pandas 0.18
DataFrame.pivot_table(data, values=None, index=None, columns=None,
                      aggfunc='mean', fill_value=None, margins=False,
                      dropna=True, margins_name='All')
```

We will first only use

> data, values=None, index=None, columns=None,

Followed by

> arggfun

Also,notice that ```fill_value``` and ```dropna``` are the same as with other Pandas methods so we will ignore this for now.

You are essentially creating a df with **index of sex** and the **columns of class** while the **values are mean of survived**.

In [13]:
titanic.pivot_table('survived',index='sex',columns='class')

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


## Multi-level pivot tables

Lets say you want the following characteristics:

- the indecies are grouped by age and sex
- the coloumns are grouped by class

You can do something like this: 

- where first you need to segment age by some range using the ```cut``` function of pandas to identify the range 
- then use this range to group the sexes and then the coloumns of classes.

In [23]:
# clean up the data with NaN's
age = titanic['age'].fillna(0)

# split the data
age = pd.cut(age,[0,18,80])

In [24]:
titanic.pivot_table('survived',index = ['sex',age],columns='class')

class               First    Second     Third
sex    age                                   
female (0, 18]   0.909091  1.000000  0.511628
       (18, 80]  0.972973  0.900000  0.423729
male   (0, 18]   0.800000  0.600000  0.215686
       (18, 80]  0.375000  0.071429  0.133663

## Applying quartiles

We can apply quartiles using the ```pd.qcut``` function [qcut](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html) to apply the calculations of quartiles.

Lets apply this quartiling to the fare column to identify the how much people spent.

In [26]:
fare = pd.qcut(titanic.fare,2)
titanic.pivot_table('survived',index = ['sex',age],columns=[fare,'class'])

fare            (-0.001, 14.454]                     (14.454, 512.329]  \
class                      First    Second     Third             First   
sex    age                                                               
female (0, 18]               NaN  1.000000  0.714286          0.909091   
       (18, 80]              NaN  0.880000  0.444444          0.972973   
male   (0, 18]               NaN  0.000000  0.260870          0.800000   
       (18, 80]              0.0  0.098039  0.125000          0.391304   

fare                                 
class              Second     Third  
sex    age                           
female (0, 18]   1.000000  0.318182  
       (18, 80]  0.914286  0.391304  
male   (0, 18]   0.818182  0.178571  
       (18, 80]  0.030303  0.192308

In [93]:
heirchicalTitanic = titanic.pivot_table('survived',index = ['sex',age],columns=[fare,'class'])

In [94]:
type(heirchicalTitanic)

pandas.core.frame.DataFrame

In [95]:
heirchicalTitanic

fare            (-0.001, 14.454]                     (14.454, 512.329]  \
class                      First    Second     Third             First   
sex    age                                                               
female (0, 18]               NaN  1.000000  0.714286          0.909091   
       (18, 80]              NaN  0.880000  0.444444          0.972973   
male   (0, 18]               NaN  0.000000  0.260870          0.800000   
       (18, 80]              0.0  0.098039  0.125000          0.391304   

fare                                 
class              Second     Third  
sex    age                           
female (0, 18]   1.000000  0.318182  
       (18, 80]  0.914286  0.391304  
male   (0, 18]   0.818182  0.178571  
       (18, 80]  0.030303  0.192308

In [96]:
heirchicalTitanic.index

MultiIndex([('female',  (0, 18]),
            ('female', (18, 80]),
            (  'male',  (0, 18]),
            (  'male', (18, 80])],
           names=['sex', 'age'])

In [97]:
heirchicalTitanic.columns

MultiIndex([( (-0.001, 14.454],  'First'),
            ( (-0.001, 14.454], 'Second'),
            ( (-0.001, 14.454],  'Third'),
            ((14.454, 512.329],  'First'),
            ((14.454, 512.329], 'Second'),
            ((14.454, 512.329],  'Third')],
           names=['fare', 'class'])

What if we want to only know the following set of conditions:

- sex = 'female'
- age: (0,18]
- fare: (14.454,512.329)
- class: 'Third'

then we would have to first row index, the coloumn and then coloumn again to access more and more information.  We would expect this number to be **0.318182**

I followed this [this](https://www.youtube.com/watch?v=tcRGa2soc-c) youtuber for examples of multilayer indexing.

Notice that we are using ```slice(None)```

In [178]:
heirchicalTitanic.loc[('female',1),(slice(None),'Third')]

fare               class
(-0.001, 14.454]   Third    0.714286
(14.454, 512.329]  Third    0.318182
Name: (female, (0, 18]), dtype: float64

## Aggregate Functions

Notice that the ```aggfunc``` has the default ```'mean'``` but we can use the other types of aggregate functions that are built in such as:

- sum
- mean
- count
- min
- max

OR

our own built in function similar to the aggregate function of pandas

Additionally, it can be specified as a dictionary mapping a column to any of the above desired options

Lets assume we want to create a pivot table for fare and survied but with different types of aggfunctions to different columns. 

In [185]:
aggfuncDictionary={'survived':'sum',
                 'fare':'mean'}

titanic.pivot_table(index='sex',
             columns='class',
             aggfunc=aggfuncDictionary)

fare                       survived             
class        First     Second      Third    First Second Third
sex                                                           
female  106.125798  21.970121  16.118810       91     70    72
male     67.226127  19.741782  12.661633       45     17    47

Notice also here that we've **omitted the values keyword**; when **specifying a mapping for aggfunc, this is determined automatically**.  So in the above cases it is assumed that there are two values one will be fare which is calulated by using the sum function while on the other is survived using the mean function.

Finally to add totals at this point you can do add ```margins``` for each of the totals

In [193]:
titanic.pivot_table(values='survived',
                    index='sex',
                    columns='class',
                    margins=True)

class,First,Second,Third,All
sex,,,,
female,0.968085,0.921053,0.500000,0.742038
male,0.368852,0.157407,0.135447,0.188908
All,0.629630,0.472826,0.242363,0.383838
